In [1]:
import sys
import talib as talib
import pandas as pd
import numpy as np
from pyecharts import options as opts
sys.path.append("..") 
from imp import reload
from src.stock import Stock
from src.chart import Chart

stock = Stock();
chart = Chart();


/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [2]:
sma_periods = [8,17,25,99,145,418,453]
keyword = '全志科技'
start_day='20120101'
stock_info=stock.get_stock_info(keyword)
hs= stock.get_daily_data(keyword,start_day,'')
hs=hs[['trade_date','close','open','high','low','vol']]
kdatas = np.array(hs.loc[:,['open','close','low','high']])
trade_dates = np.array(hs.trade_date)
vols = np.array(hs['vol'])
hs['sar'] = talib.SAR(hs.high, hs.low, acceleration=0.1, maximum=2)

sma_dic= {}
for period in sma_periods:
    name = "SMA" + str(period)
    sma = talib.SMA(np.array(hs.close),period)
    hs[name] =sma;
    sma_dic[name]={}
    sma_dic[name]['name'] = name;
    sma_dic[name]['datas'] = sma.round(2).tolist();
sar = {}
sar['down'] = hs.loc[hs['close'] > hs['sar'],['trade_date','sar']]
sar['up'] = (hs[hs['close'] < hs['sar']])[['trade_date','sar']]

grid_chart= chart.get_grid(stock_info,trade_dates.tolist(), kdatas.tolist(),sma_dic,vols.tolist(), sar=sar)
    
hs['macd_dif'],hs['macd_dem'],hs['macd_ocr'] = talib.MACD(hs.close,12,26,9)

macd = chart.get_macd_chart(
    trade_dates.tolist(),
    hs.macd_dif.to_numpy().round(2).tolist(),
    hs.macd_dem.to_numpy().round(2).tolist(),
    (hs["macd_ocr"]*2).to_numpy().round(2).tolist()
)

grid_chart.add(
    macd,
    grid_opts=opts.GridOpts(
        pos_left="6%", pos_right="8%", pos_top="75%", height="10%"
    ),
)


['_id', 'low', 'vol', 'high', 'open', 'close', 'amount', 'pct_chg', 'ts_code', 'creator', 'modifier', 'pre_close', 'gmt_create', 'is_deleted', 'trade_date', 'gmt_modified']


In [ ]:
grid_chart.render_notebook()
